In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns
# from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format
random_state = 42
from datetime import date
import plotly.graph_objects as go
from plotly import offline as pyoff



from datetime import datetime, date, time, timedelta
import calendar

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import time
from xgboost import plot_importance
from sklearn.metrics import roc_auc_score

from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb


In [3]:
###Vamos a hacer una estimacion del exito de la campaña. Como no tenemos un historico de campañas, esstimamos que un 30% de 
##de los clientes, pueden llegar a leerse el email

In [4]:
df = pd.read_csv('definitivo_10000clientes.csv')

In [5]:
df

,pk_cid,recency,ingesta,frequency,monetary_value,r_quartile,f_quartile,m_quartile,i_quartile,RFMScore,region_code,salary,age,gender,unidades_vendidas_total,em_acount,debit_card
0,1374819,505,0,17,"4,290.00",4,1,1,4,411,28.00,"121,839.00",38.00,2.00,8.00,0.00,1.00
1,1392320,408,0,14,"3,190.00",4,1,1,4,411,28.00,"107,353.08",27.00,1.00,7.00,1.00,1.00
2,1377496,492,0,17,"2,880.00",4,1,1,4,411,28.00,"101,396.49",28.00,1.00,6.00,0.00,1.00
3,1374123,508,0,17,"2,850.00",4,1,1,4,411,28.00,"113,652.32",52.00,2.00,6.00,0.00,1.00
4,1374403,506,0,17,"2,840.00",4,1,1,4,411,28.00,"90,748.62",51.00,1.00,6.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11900,1379212,483,0,16,170.00,4,1,1,4,411,4.00,"84,976.65",50.00,2.00,0.50,0.00,0.00
11901,1374521,506,0,17,170.00,4,1,1,4,411,8.00,"121,655.40",30.00,1.00,1.00,1.00,0.00
11902,1374515,506,0,16,170.00,4,1,1,4,411,10.00,"80,420.01",43.00,2.00,0.00,0.00,0.00
11903,1401449,350,0,11,170.00,4,1,1,4,411,37.00,"47,390.46",40.00,2.00,1.00,1.00,0.00


In [6]:
df["salary"] = df["salary"].astype(int)

df["age"] = df["age"].astype(int)

df["gender"] = df["gender"].astype(int)


df["monetary_value"] = df["monetary_value"].astype(int)

In [9]:
##como en este ejemplo, son clientes del año 2018, pero que supusieron unas ventas muy superiores
##a las del 2019, vamos a suponer, que como hace mas de 5 meses desde la ultima compra, solo,mostraran
## un cierto interes, un 10% de los clientes a los que enviamos un email(con clientes mas recientes,
###,habiamos contemplado un interés de un 30%, ya que es mas cercana en el tiempo, su compra)

In [10]:
df_30= df.sample(n=1300, random_state=42)

In [11]:
##Lo contemplamos como un modelo supervisado por clasificacion, porque queremos saber, de esa estimacion del 30% de clientes
##que van a tener interes despues de recibir el email, que porcentaje, va a comprar o no, y por tanto ,generara ingresos, o no.

In [12]:
df_30

,pk_cid,recency,ingesta,frequency,monetary_value,r_quartile,f_quartile,m_quartile,i_quartile,RFMScore,region_code,salary,age,gender,unidades_vendidas_total,em_acount,debit_card
6363,1383985,459,0,16,560,4,1,1,4,411,28.00,108144,31,1,2.00,1.00,0.00
1078,1387430,442,0,15,1180,4,1,1,4,411,50.00,113652,37,2,3.00,0.00,1.00
2145,1390773,417,0,14,980,4,1,1,4,411,41.00,111104,34,2,2.00,1.00,1.00
673,1378121,487,0,17,1360,4,1,1,4,411,24.00,111329,51,2,3.00,1.00,0.00
6768,1375240,504,0,17,530,4,1,1,4,411,41.00,361168,20,2,1.00,1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2929,1390243,423,0,14,860,4,1,1,4,411,29.00,54152,37,2,2.00,1.00,1.00
7297,1400026,360,0,12,480,4,1,1,4,411,50.00,88595,20,2,1.50,1.00,0.50
10872,1413077,309,0,11,220,4,1,1,4,411,11.00,128083,22,1,1.00,1.00,0.00
8752,1389659,428,0,15,330,4,1,1,4,411,36.00,110007,43,2,1.00,1.00,0.00


In [13]:
###Para este dataset, vamos a predecir el incremento de revenue, que puede haber.

In [14]:
y= df_30['monetary_value']

In [15]:
X= df_30

In [16]:
X= X.drop('monetary_value', axis=1)

In [17]:
from sklearn.model_selection import train_test_split
random_seed = 42
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)
                                                    


In [18]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer as mlb


In [19]:
from lightgbm import LGBMModel,LGBMClassifier

In [20]:
t0 = time.time()
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train,y_train)
y_pred = lgbm_model.predict(X_test)
proba = lgbm_model.predict_proba(X_test)

cv_score = cross_val_score(lgbm_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
lgbm_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.06293706293706294
CV Score : 0.07806896551724138
              precision    recall  f1-score   support

         170       0.30      0.30      0.30        10
         180       0.00      0.00      0.00        10
         190       0.00      0.00      0.00         7
         200       0.00      0.00      0.00         3
         210       0.00      0.00      0.00         7
         220       0.25      0.12      0.17         8
         230       0.05      0.08      0.06        12
         240       0.05      0.25      0.08         4
         250       0.08      0.12      0.10         8
         260       0.15      0.29      0.20         7
         270       0.00      0.00      0.00         6
         280       0.00      0.00      0.00         9
         290       0.21      0.30      0.25        10
         300       0.00      0.00      0.00         5
         310       0.00      0.00      0.00         5
         320       0.00      0.00      0.00         5
         330       

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [21]:
t0 = time.time()
random_model = RandomForestClassifier()
random_model.fit(X_train,y_train)
y_pred = random_model.predict(X_test)
proba = random_model.predict_proba(X_test)

cv_score = cross_val_score(random_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
random_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.09090909090909091
CV Score : 0.09758949096880132
              precision    recall  f1-score   support

         170       0.38      0.50      0.43        10
         180       0.00      0.00      0.00        10
         190       0.33      0.14      0.20         7
         200       0.17      0.33      0.22         3
         210       0.06      0.14      0.09         7
         220       0.00      0.00      0.00         8
         230       0.00      0.00      0.00        12
         240       0.06      0.25      0.10         4
         250       0.08      0.12      0.10         8
         260       0.15      0.29      0.20         7
         270       1.00      0.00      0.00         6
         280       0.10      0.11      0.11         9
         290       0.27      0.40      0.32        10
         300       0.00      0.00      0.00         5
         310       0.00      0.00      0.00         5
         320       1.00      0.00      0.00         5
         330       

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
t0 = time.time()
decission_model = DecisionTreeClassifier()
decission_model.fit(X_train,y_train)
y_pred =decission_model.predict(X_test)
proba =decission_model.predict_proba(X_test)

cv_score = cross_val_score(decission_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
decission_model

Accuracy : 0.06526806526806526
CV Score : 0.07004269293924466
              precision    recall  f1-score   support

         170       0.33      0.30      0.32        10
         180       0.00      0.00      0.00        10
         190       0.00      0.00      0.00         7
         200       0.00      0.00      0.00         3
         210       0.08      0.14      0.10         7
         220       0.00      0.00      0.00         8
         230       0.00      0.00      0.00        12
         240       0.00      0.00      0.00         4
         250       0.12      0.12      0.12         8
         260       0.13      0.29      0.18         7
         270       0.50      0.17      0.25         6
         280       0.09      0.11      0.10         9
         290       0.17      0.20      0.18        10
         300       0.00      0.00      0.00         5
         310       0.00      0.00      0.00         5
         320       1.00      0.00      0.00         5
         330       

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [23]:
t0 = time.time()
xgbc_model = XGBClassifier()
xgbc_model.fit(X_train,y_train)
y_pred =xgbc_model.predict(X_test)
proba =xgbc_model.predict_proba(X_test)

cv_score = cross_val_score(xgbc_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
xgbc_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.07226107226107226
CV Score : 0.0700361247947455
              precision    recall  f1-score   support

         170       0.50      0.40      0.44        10
         180       0.00      0.00      0.00        10
         190       0.00      0.00      0.00         7
         200       0.00      0.00      0.00         3
         210       0.00      0.00      0.00         7
         220       0.14      0.12      0.13         8
         230       0.06      0.08      0.07        12
         240       0.07      0.25      0.11         4
         250       0.11      0.12      0.12         8
         260       0.15      0.29      0.20         7
         270       0.00      0.00      0.00         6
         280       0.00      0.00      0.00         9
         290       0.27      0.30      0.29        10
         300       0.00      0.00      0.00         5
         310       0.00      0.00      0.00         5
         320       0.00      0.00      0.00         5
         330       0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [24]:
##El mejor modelo es el RandomForest, que pronóstica, que alrededor del 9% de clientes, interesados, acabaran comprando.

In [25]:
df_30['monetary_value'].sum()

840970

In [ ]:
df_30

In [27]:
plot_data = [
    go.Bar(
        x=df_30['recency'],
        y=df_30['monetary_value']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Rec_Monetary año2018'
    )
plt.figure(figsize=(10,10))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig810rec-monetary.html')
pyoff.iplot(fig)

<Figure size 720x720 with 0 Axes>

In [33]:
##Los ingresos previstos, son de 50458€ más a nuestra facturación del ultimo mes, 
###teniendo en cuenta la prevision de que venderemos a un 6%

In [35]:
###Habiendo mirado que el coste que tenemos por enviar un email a 10000 clientes ,es de 120€, el ROI es de un 41948%

In [28]:
df_30['em_acount'].sum()

880.5

In [29]:
df_30['debit_card'].sum()

485.0

In [30]:
df_30['unidades_vendidas_total'].sum()

2409.5

In [31]:
plot_data = [
    go.Bar(
        x=df_30['recency'],
        y=df_30['em_acount']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='em_acount compran max 90 dias clientes con interes'
    )
plt.figure(figsize=(10,10))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig811em_account_unidades10000.html')
pyoff.iplot(fig)

<Figure size 720x720 with 0 Axes>

In [32]:
plot_data = [
    go.Bar(
        x=df_30['recency'],
        y=df_30['debit_card']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='debit_card ultimos 90 dias clientes con interes'
    )
plt.figure(figsize=(10,10))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig812debit_card_unidades10000.html')
pyoff.iplot(fig)

<Figure size 720x720 with 0 Axes>

In [34]:
##Teniendo en cuenta, que según nuestro modelo, tenemos previsto vender unas 2410 unidades mas de todo nuestro portforlio 
##de producto, durante los 30días de campaña, de los que unos 485 serán un producto similar a
##debit card, y 880 a uno similar a em account (sólo 2 nuevos productos ,similares a em_account
## y debit card, de los 14
##productos que tenemos,van a reprentar el 50% de las unidades vendidas en la campaña), ésta 
##consistiría, en informar  de la existencia de nuestro portfolio, asi como en promocionar  
##3 nuevos productos: debit_card_puntos a 50€, em_acount_nomina a 10€ y uno de inversión 
##a un precio de lanzamiento
##de cada transacción de 30, en lugar de 40€: hemos firmado un acuerdo con nuestro partner,
##para distribuir un producto de 
##inversion asociado a un valor refugio, como es el oro, que actualmente puede tener mucho 
##recorrido , y que denominamos 
##Gold_Found, con un objetivo de venta durante la campaña de unas 350 transacciones, que nos de
## un añadido de 1050€, a la estimación de 50500€. Teniendo en cuenta, que en el ultimo mes
##de 2019, hemos facturado 19550€ ,incrementariamos un 350% nuestra facturacion del mes de junio,
###hasta los 70000€
##que en relacion a 2018, sería un 8% de lo facturado en junio, pero mejor, que el 3%, que es la
##la tendencia actual, sin campañas.

In [36]:
###si supusieramos que el 100% de los clientes a los que enviamos el email, estuvieran interesados
##, qué sucederia?

In [38]:
y= df['monetary_value']

In [39]:
X= df

In [40]:
X= X.drop('monetary_value', axis=1)

In [41]:
from sklearn.model_selection import train_test_split
random_seed = 42
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)

In [42]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer as mlb

In [43]:
from lightgbm import LGBMModel,LGBMClassifier

In [44]:
t0 = time.time()
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train,y_train)
y_pred = lgbm_model.predict(X_test)
proba = lgbm_model.predict_proba(X_test)

cv_score = cross_val_score(lgbm_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
lgbm_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.02061593280733011
CV Score : 0.019683456289626887
              precision    recall  f1-score   support

         170       1.00      0.00      0.00        84
         180       1.00      0.00      0.00        63
         190       0.00      0.00      0.00        59
         200       1.00      0.00      0.00        59
         210       1.00      0.00      0.00        71
         220       1.00      0.00      0.00        52
         230       0.02      0.99      0.04        80
         240       0.00      0.00      0.00        71
         250       1.00      0.00      0.00        97
         260       0.00      0.00      0.00        73
         270       1.00      0.00      0.00        53
         280       1.00      0.00      0.00        84
         290       1.00      0.00      0.00        70
         300       1.00      0.00      0.00        38
         310       1.00      0.00      0.00        38
         320       1.00      0.00      0.00        35
         330      

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [45]:
t0 = time.time()
random_model = RandomForestClassifier()
random_model.fit(X_train,y_train)
y_pred = random_model.predict(X_test)
proba = random_model.predict_proba(X_test)

cv_score = cross_val_score(random_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
random_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.13031305675744465
CV Score : 0.1241229248670265
              precision    recall  f1-score   support

         170       0.46      0.52      0.49        84
         180       0.15      0.16      0.15        63
         190       0.02      0.02      0.02        59
         200       0.09      0.08      0.09        59
         210       0.14      0.18      0.16        71
         220       0.06      0.08      0.07        52
         230       0.18      0.21      0.19        80
         240       0.19      0.18      0.18        71
         250       0.16      0.22      0.19        97
         260       0.20      0.22      0.21        73
         270       0.14      0.17      0.16        53
         280       0.21      0.29      0.24        84
         290       0.17      0.17      0.17        70
         300       0.17      0.18      0.18        38
         310       0.07      0.08      0.08        38
         320       0.04      0.03      0.03        35
         330       0

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [46]:
t0 = time.time()
decission_model = XGBClassifier()
decission_model.fit(X_train,y_train)
y_pred =decission_model.predict(X_test)
proba =decission_model.predict_proba(X_test)

cv_score = cross_val_score(decission_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
decission_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.1150419954186816
CV Score : 0.10456391763106827
              precision    recall  f1-score   support

         170       0.40      0.40      0.40        84
         180       0.15      0.16      0.15        63
         190       0.07      0.07      0.07        59
         200       0.10      0.10      0.10        59
         210       0.11      0.17      0.13        71
         220       0.06      0.06      0.06        52
         230       0.13      0.21      0.16        80
         240       0.25      0.25      0.25        71
         250       0.20      0.35      0.26        97
         260       0.16      0.19      0.17        73
         270       0.08      0.13      0.10        53
         280       0.20      0.32      0.25        84
         290       0.12      0.13      0.12        70
         300       0.13      0.11      0.12        38
         310       0.03      0.03      0.03        38
         320       0.11      0.06      0.08        35
         330       0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [47]:
t0 = time.time()
xgbc_model = DecisionTreeClassifier()
xgbc_model.fit(X_train,y_train)
y_pred =xgbc_model.predict(X_test)
proba =xgbc_model.predict_proba(X_test)

cv_score = cross_val_score(xgbc_model,X_train,y_train,cv=5).mean()
score = accuracy_score(y_test,y_pred)
bin_clf_rep = classification_report(y_test,y_pred, zero_division=1)


print("Accuracy :", score)
print("CV Score :", cv_score)
print(bin_clf_rep)
print(confusion_matrix(y_test,y_pred))
print("Time Taken :", time.time()-t0, "seconds")
xgbc_model

C:\Users\evive\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=5.



Accuracy : 0.0847543904301349
CV Score : 0.08989487826148443
              precision    recall  f1-score   support

         170       0.40      0.33      0.36        84
         180       0.07      0.06      0.07        63
         190       0.08      0.07      0.07        59
         200       0.09      0.08      0.09        59
         210       0.08      0.08      0.08        71
         220       0.04      0.04      0.04        52
         230       0.09      0.10      0.10        80
         240       0.20      0.18      0.19        71
         250       0.15      0.15      0.15        97
         260       0.10      0.08      0.09        73
         270       0.12      0.13      0.12        53
         280       0.16      0.17      0.16        84
         290       0.10      0.10      0.10        70
         300       0.07      0.11      0.09        38
         310       0.03      0.05      0.04        38
         320       0.03      0.03      0.03        35
         330       0

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [50]:
df_ventas= df.groupby(by=['pk_cid'],as_index=False)["monetary_value"].sum()


In [51]:
df_ventas.sum()

pk_cid            16228612505
monetary_value        7733860
dtype: int64

In [52]:
###Si el 100% de los clientes ,estuviera interesado, comprarian un 13%, según el mejor resultado 
## que da Random Forest, y supondría unos ingresos durante la campaña del mes de junio , de 
### 1millon €, situacion, que obviamente, no se va a dar, pero que no podemos extrapolar,ese 
###resultado a un mes, dividiendo por 12.

In [53]:
plot_data = [
    go.Bar(
        x=df_30['salary'],
        y=df_30['monetary_value']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='ingresos segun salario 10000-10'
    )
plt.figure(figsize=(10,10))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig700ingresos_salary30de10000-10.html')
pyoff.iplot(fig)

<Figure size 720x720 with 0 Axes>

In [54]:
plot_data = [
    go.Bar(
        x=df_30['age'],
        y=df_30['monetary_value']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='ingresos segun la edad10000-10'
    )
plt.figure(figsize=(10,10))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig600ingresos_age10000-10.html')
pyoff.iplot(fig)

<Figure size 720x720 with 0 Axes>

In [55]:
plot_data = [
    go.Bar(
        x=df_30['region_code'],
        y=df_30['monetary_value']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='ingresos segun la regioncode10000-10'
    )
plt.figure(figsize=(10,10))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig600ingresos_region10000-10.html')
pyoff.iplot(fig)

<Figure size 720x720 with 0 Axes>

In [ ]:
###Vamos a analizar respecto los 10000clientes

In [56]:
plot_data = [
    go.Bar(
        x=df['recency'],
        y=df['monetary_value']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='ingresos segun la recency'
    )
plt.figure(figsize=(10,10))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig600ingresos_recency10000.html')
pyoff.iplot(fig)

<Figure size 720x720 with 0 Axes>

In [57]:
plot_data = [
    go.Bar(
        x=df['salary'],
        y=df['monetary_value']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='ingresos segun la salary10000'
    )
plt.figure(figsize=(10,10))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig600ingresos_recency10000.html')
pyoff.iplot(fig)

<Figure size 720x720 with 0 Axes>

In [58]:
plot_data = [
    go.Bar(
        x=df['age'],
        y=df['monetary_value']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='ingresos segun la edad10000'
    )
plt.figure(figsize=(10,10))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig600ingresos_recency10000.html')
pyoff.iplot(fig)

<Figure size 720x720 with 0 Axes>

In [60]:
plot_data = [
    go.Bar(
        x=df['region_code'],
        y=df['monetary_value']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='ingresos segun la region10000'
    )
plt.figure(figsize=(10,10))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig600ingresos_recency10000.html')
pyoff.iplot(fig)

<Figure size 720x720 with 0 Axes>